In [6]:
import collections
import datetime
import requests
import inflection
import pandas
import pytz

In [7]:
pandas.set_option('max_rows', None)
pandas.set_option('max_columns', None)

In [8]:
STALE_DT = (datetime.datetime.utcnow() - datetime.timedelta(minutes=90)).replace(tzinfo=pytz.utc)

In [9]:
STALE_DT

datetime.datetime(2016, 11, 8, 23, 8, 31, 955424, tzinfo=<UTC>)

In [10]:
def strip_updated(updated_at):
    updated_at = updated_at.replace('CT<br /> ', '')
    dt = datetime.datetime.strptime(updated_at, '%H:%M:%S %d %b %Y')
    dt = dt.replace(tzinfo=pytz.timezone('US/Central'))
    dt = dt.astimezone(pytz.utc)
    return dt

In [11]:
# url = 'http://www.cmegroup.com/CmeWS/mvc/Quotes/Option/2916/G/X6/Active?optionProductId=2915&strikeRange=Active&pageSize=50&_=1478643918531'
url = 'https://query1.finance.yahoo.com/v7/finance/options/SPY?formatted=true&crumb=oPWCX4Hh1ne&lang=en-US&region=US&straddle=false&corsDomain=finance.yahoo.com'

In [12]:
req = requests.get(url)

In [13]:
data = req.json()

In [14]:
# line_cols = ['strikePrice', 'strikeRank', 'underlyingFutureContract']

# strikes_list = []
# for strike_line in data['optionContractQuotes']:
#     to_append = {inflection.underscore(c): strike_line[c] for c in line_cols}
    
#     to_append.update({'put__{c}'.format(c=c): v for c, v in strike_line['put'].items()})
#     to_append.update({'call__{c}'.format(c=c): v for c, v in strike_line['call'].items()})
    
#     to_append['put__updated'] = strip_updated(to_append['put__updated'])
#     to_append['call__updated'] = strip_updated(to_append['call__updated'])
    
#     strikes_list.append(to_append)
# strikes_df = pandas.DataFrame(strikes_list)

In [15]:
# strikes_df['strike_price'] = strikes_df['strike_price'].astype(float)
# strikes_df['call__last'] = strikes_df['call__last'].apply(lambda k: float(k) if k != '-' else None)
# strikes_df['put__last'] = strikes_df['put__last'].apply(lambda k: float(k) if k != '-' else None)

In [16]:
# non_stale_slice = strikes_df['call__updated'] > STALE_DT
# mod_1000_strike = strikes_df['strike_price'] % 1000 == 0
# within_fwd      = strikes_df['strike_price'].apply(lambda k: abs(k-214000) < 8000)

In [17]:
# calls = strikes_df[non_stale_slice & mod_1000_strike & within_fwd][['strike_price','call__last','call__updated']]
# calls['call__spread'] = calls['call__last'].diff(-1)
# calls['call__fly'] = calls['call__spread'].diff(-1)
# calls['fly_imp_prob'] = calls['call__fly'] / 10

In [18]:
opts = data['optionChain']['result'][0]['options'][0]

In [19]:
MIN_STRIKE = int(min(min(opt['strike']['raw'] for opt in opts['calls']), min(opt['strike']['raw'] for opt in opts['puts'])))
MAX_STRIKE = int(max(max(opt['strike']['raw'] for opt in opts['calls']), max(opt['strike']['raw'] for opt in opts['puts'])))

In [20]:
MIN_STRIKE, MAX_STRIKE

(175, 235)

In [21]:
calls = {}
puts  = {}

for call_dict in opts['calls']:
    calls[call_dict['strike']['raw']] = call_dict
for puts_dict in opts['puts']:
    puts[puts_dict['strike']['raw']] = puts_dict

In [22]:
options_list = []
for k in range(MIN_STRIKE, MAX_STRIKE+1, 1):
    call_dict = calls.get(k, None)
    put_dict = puts.get(k, None)
    
    to_append = {}
    
    if call_dict:
        to_append.update({
            'call_strike': call_dict['strike']['raw'],
            'call_expiration': datetime.datetime.fromtimestamp(call_dict['expiration']['raw']),
            'call_ask': call_dict['ask']['raw'],
            'call_bid': call_dict['bid']['raw'],
            'call_mm': (call_dict['ask']['raw'] + call_dict['bid']['raw']) / 2,
            'call_symbol': call_dict['contractSymbol'],
            'call_open_interest':  call_dict['openInterest']['raw'],
        })
    
    if put_dict:
        to_append.update({
            'put_strike': put_dict['strike']['raw'],
            'put_expiration': datetime.datetime.fromtimestamp(put_dict['expiration']['raw']),
            'put_ask': put_dict['ask']['raw'],
            'put_bid': put_dict['bid']['raw'],
            'put_mm': (put_dict['ask']['raw'] + put_dict['bid']['raw']) / 2,
            'put_symbol': put_dict['contractSymbol'],
            'put_open_interest': put_dict['openInterest']['raw'],
        })
    options_list.append(to_append)
df = pandas.DataFrame(options_list)

In [23]:
STRIKE_LIST = list(range(MIN_STRIKE, MAX_STRIKE+1, 1))

In [24]:
df = df[df['call_strike'].isin(STRIKE_LIST)].reset_index(drop=True)

In [25]:
df['call_spread'] = df['call_mm'] - df['call_mm'].shift(-1)
df['call_fly']    = df['call_spread'] - df['call_spread'].shift(-1)

df['put_spread'] = df['put_mm'] - df['put_mm'].shift(-1)
df['put_fly']    = df['put_spread'] - df['put_spread'].shift(-1)

In [26]:
df['fly'] = df.apply(lambda k: k['call_fly'] if k['call_strike'] > 215 else k['put_fly'], axis=1).round(decimals=4)

In [31]:
df[['fly','call_strike','call_fly','call_mm','put_strike','put_fly','put_mm','call_expiration','put_expiration']]

,fly,call_strike,call_fly,call_mm,put_strike,put_fly,put_mm,call_expiration,put_expiration
0,0.000,175.0,2.980,33.535,175.0,0.000000e+00,0.005,2016-11-08 16:00:00,2016-11-08 16:00:00
1,0.020,180.0,-0.485,28.560,180.0,2.000000e-02,0.005,2016-11-08 16:00:00,2016-11-08 16:00:00
2,0.040,182.0,-6.850,26.565,182.0,4.000000e-02,0.005,2016-11-08 16:00:00,2016-11-08 16:00:00
3,-0.050,190.0,13.705,24.085,190.0,-5.000000e-02,0.025,2016-11-08 16:00:00,2016-11-08 16:00:00
4,0.005,194.0,-10.660,14.755,194.0,5.000000e-03,0.085,2016-11-08 16:00:00,2016-11-08 16:00:00
5,0.000,195.0,10.615,19.130,195.0,0.000000e+00,0.095,2016-11-08 16:00:00,2016-11-08 16:00:00
6,0.005,196.0,-5.320,12.845,196.0,5.000000e-03,0.110,2016-11-08 16:00:00,2016-11-08 16:00:00
7,0.015,197.0,-5.090,17.175,197.0,1.500000e-02,0.125,2016-11-08 16:00:00,2016-11-08 16:00:00
8,0.000,198.0,10.135,16.185,198.0,2.775558e-17,0.145,2016-11-08 16:00:00,2016-11-08 16:00:00
9,0.020,199.0,-4.910,10.105,199.0,2.000000e-02,0.180,2016-11-08 16:00:00,2016-11-08 16:00:00
